In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
import torchviz
from torchviz import make_dot
from torch.optim import Adam
import graphviz
import tqdm
import os
import numpy

BATCH_SIZE = 40
LEARNING_RATE = 0.001
EPOCHS = 20
LSTM_NODES = 256
NUM_SENTENCES = 40
SENTENCE_LENGTH = 98
VOCAB_SIZE = 49152
EMBEDDING_SIZE = 4096
NUM_EPOCHS = 20





torch.manual_seed(691)

# TODO: Model checkpoints, Model saving
try:
    pretrained_weights = torch.load(r'C:\Users\Andrew\Desktop\nasa_project\Juliet\Large Files\aix3-7b-base (1).pt')
    print("Weights loaded successfully.")
except Exception as e:
    print(f"Failed to load weights: {e}")

# Once you've identified the key for the embeddings, you can extract them like this:
word_vectors = pretrained_weights['tok_embeddings.weight']
print(word_vectors.shape)


dummy_input = torch.randint(0, VOCAB_SIZE, (BATCH_SIZE, SENTENCE_LENGTH, NUM_SENTENCES))

class LSTMClassifier(nn.Module):    
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, batch_first, bidirectional, dropout, pretrained_weights, batch_size, sentence_length):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)
        self.embedding.weight.data.copy_(pretrained_weights)

    def forward(self, text):
        
        batch_size, sentence_length, num_sentences = text.size()
        text = text.view(batch_size, sentence_length * num_sentences)
        
        embedded = self.dropout(self.embedding(text))
        #print(f"Shape after embedding: {embedded.shape}")
        lstm_output, (hidden, _) = self.rnn(embedded)
        # print(f"Shape after LSTM: {lstm_output.shape}")
        pooled = torch.mean(lstm_output, dim=1)
        # Pass through the fully connected layer
        output = self.fc(self.dropout(pooled))
       
        output = torch.sigmoid(output)
        # print(f"Shape of final output: {output.shape}")
        
        return output

Juliet_Max_Pool = LSTMClassifier(
    vocab_size=VOCAB_SIZE,
    embedding_dim=EMBEDDING_SIZE,  
    hidden_dim=LSTM_NODES,
    output_dim=40,  
    n_layers=2,
    batch_first=True,
    bidirectional=True,
    dropout=0.5,
    pretrained_weights=word_vectors,
    batch_size=BATCH_SIZE,
    sentence_length=SENTENCE_LENGTH
)
print(Juliet_Max_Pool)

model_output = Juliet_Max_Pool(dummy_input)

# Visualize the model
# dot = make_dot(model_output, params=dict(Juliet_Max_Pool.named_parameters()))


# To render and view the graph (requires Graphviz installed in your system)
#dot.render('max_pool_model_visualization', format='png', cleanup=True)


Weights loaded successfully.
torch.Size([49152, 4096])
LSTMClassifier(
  (embedding): Embedding(49152, 4096)
  (rnn): LSTM(4096, 256, num_layers=2, dropout=0.5, bidirectional=True)
  (fc): Linear(in_features=512, out_features=40, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [5]:
from torchview import draw_graph
import graphviz



model_graph = draw_graph(
    model=Juliet_Max_Pool,  
    input_data=dummy_input,  
    graph_dir='LR',
    depth=3,
    graph_name='Juliet_Max_Pool',
    hide_module_functions=True,
    roll=True
)


model_graph.visual_graph
model_graph
model_graph.resize_graph(scale=5.0) # scale as per the view model_graph.visual_graph.render(format='svg') 
model_graph.visual_graph.render(format='png')


(dot.exe:17008): Pango-WARNING **: couldn't load font "Linux libertine Not-Rotated 10", falling back to "Sans Not-Rotated 10", expect ugly output.


'Juliet_Max_Pool.gv.png'